In [1]:
import numpy as np
import pandas as pd
import spacy
import nltk 
import requests
import json
import googletrans
import random

from random import randint
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.lang.en import English
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet 
from googletrans import Translator

spacy.load('en_core_web_sm')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

analyser = SentimentIntensityAnalyzer()
parser = English()
porter = PorterStemmer()
translator = Translator()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ninagroot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
file = 'edit-db_model2.xlsx'
df_gratitude = pd.read_excel(file, 'gratitude')
#df_exercises = pd.read_excel('exercises_model2.xlsx')
df_journal = pd.read_excel('prepared_journal_model2_lang.xlsx')
df_journal = df_journal.dropna(subset = ['id'])

topics = {
    'social media': ['social', 'media', 'network', 'twitter', 'facebook', 'instagram', 'web', 'linkedin', 'google', 'viral', 'google+', 'event', 'website', 'reach', 'telegraph', 'telephone', 'blog', 'internet', 'social', 'sociality', 'technoself', 'intranet', 'cyber', 'business', 'sociable', 'activist', 'microblogging', 'socially', 'cybernetic', 'pinterest', 'youtube', 'tumblr', 'cyberspace', 'extranet', 'telnet', 'cybernetics', 'modem', 'communication', 'informatics', 'virtual', 'party', 'picture'],
    'body insecurity': ['body insecurity', 'body', 'insecure', 'belly', 'skin', 'waist', 'hair', 'eyes', 'lips', 'ears', 'forehead', 'eyebrow', 'chin', 'scar', 'nose', 'look', 'eat', 'ate', 'boldness', 'beard', 'smile', 'teeth', 'stomach', 'back', 'ass', 'butt', 'frame', 'feature', 'physique', 'bod', 'bodies', 'torso', 'arm', 'head', 'corpse', 'chest', 'shape', 'shoulder', 'form', 'neck', 'structure', 'leg', 'mass', 'weight', 'shaky', 'unattractive', 'unsure', 'anxious', 'jealous', 'unconfident', 'distrustful', 'uncomfortable', 'unsecured', 'neurotic', 'hopeless', 'depressed', 'introverted', 'helpless', 'fragile', 'danger', 'anxiety', 'mistrust', 'instability', 'unrest', 'hardship', 'vulnerability', 'lawlessness', 'uncertainty', 'turmoil', 'strife', 'fragility', 'hopelessness', 'malnutrition', 'alienation', 'violence', 'poverty', 'tension', 'shortages', 'despair', 'chaos', 'distrust', 'resentment', 'underdevelopment', 'anarchy', 'paranoia', 'cynicism', 'fear', 'scared', 'insecure', 'fearful'],
    'social anxiety': ['social anxiety', 'shyness', 'blushing', 'fear', 'worry', 'stress', 'nervousness', 'attention', 'shame', 'panic', 'social', 'attack', 'own', 'activities', 'lonely', 'alone', 'rejection', 'distance', 'conversation', 'afraid', 'mistake', 'stupid', 'insecurity', 'careful', 'courage', 'public', 'speaking', 'nausea', 'stutter', 'acting', 'performance', 'stage', 'fright', 'fear', 'public', 'shyness', 'blushing', 'anxiety', 'self-consciousness', 'panic', 'illness', 'mood disorder', 'timidness', 'street', 'outside', 'social functioning', 'parties', 'outgoing', 'illness', 'peer rejection', 'stutter', 'stage fright', 'guilt', 'panic attack'],
    'relationships' : ['friendship', 'best', 'friend', 'classmate', 'schoolmate', 'roomie', 'boyfriend', 'sister', 'amigo', 'acquanitance', 'brother', 'comerade', 'girlfriend', 'person', 'pal', 'schoolfriend', 'mate', 'colleague', 'roommate', 'love', 'pal', 'buddy', 'parent', 'parents', 'father', 'mother', 'grandma', 'grandpa', 'talk', 'soulmate', 'fight', 'trust', 'understanding', 'happiness', 'relations', 'empathy', 'company', 'feelings', 'unity', 'friendship', 'friend', 'companionship', 'relationship', 'relation', 'child', 'kid', 'son', 'daughter', 'friendship', 'affinity', 'kinship', 'affiliation', 'association', 'voices', 'partnership', 'acquaintance', 'state', 'consanguinity', 'sisterhood', 'brotherhood', 'membership', 'human relationship', 'family relationship', 'relate', 'relational', 'interrelation', 'family', 'partner', 'relatedness', 'relatedness', 'correlate', 'parentage', 'relative', 'tie', 'account', 'involvement', 'ties', 'affair', 'engagement', 'contacts', 'rapport', 'connection', 'dealing', 'terms', 'regards', 'concerned', 'comparison', 'linkage', 'motherhood', 'maternity', 'paternity', 'fatherhood', 'subjection', 'birth', 'romance', 'life', 'understanding', 'shared', 'future', 'complicated', 'marriage', 'ally', 'feelings', 'intereseted', 'personality', 'mutual', 'personal', 'incest', 'foster', 'absence', 'bilateral', 'sense', 'resolve', 'cousin', 'business relationship', 'love affair', 'personal relation', 'friendly relationship', 'blood kinship', 'marital bed', 'line of descent', 'sexual relationship', 'marital relationship', 'phylogenetic relationship', 'aunt', 'familial', 'fraternal', 'sister', 'nephew', 'grandniece', 'analogy', 'siblinghood', 'framily', 'brother', 'heteronymous', 'neighbourship', 'granddaughter', 'nieceship', 'niece', 'kiss cousin', 'social relation', 'stepbrother', 'intimacy', 'uncle', 'spousal', 'stepsister', 'familywise', 'hookup', 'nonmarriage', 'private life', 'stepfamily', 'interaction', 'lust', 'platonic love', 'affection', 'kindness', 'pleasure', 'compassion', 'sex', 'hate', 'heart', 'emotion', 'religion', 'sexual attraction', 'passion', 'lover', 'fight', 'lie'],
    'work': ['work', 'job', 'turn', 'ferment', 'exercise', 'play', 'sour', 'work out', 'labor', 'go', 'employment', 'put to work', 'process', 'bring', 'workplace', 'toil', 'coursework', 'forge', 'make', 'labour', 'housewifery', 'service', 'exchange', 'studio', 'answer', 'workpiece', 'spadework', 'ironwork', 'leatherwork', 'lacework', 'employ', 'metalwork', 'paperwork', 'nightwork', 'writing', 'welfare work', 'lacquerware', 'learning', 'masterpiece', 'silverwork', 'task', 'workload', 'crop', 'operate', 'function', 'wreak', 'work on', 'act', 'puzzle out', 'make for', 'piece of work', 'do work', 'project', 'deal', 'work ethic', 'deal', 'exploit', 'overwork', 'worker', 'housekeeping', 'business', 'task', 'employment', 'vocation', 'money', 'subcontract', 'profession', 'career', 'employer', 'place', 'duty', 'workplace', 'hire', 'internship', 'problem', 'chore', 'volunteer', 'position', 'laborer', 'handyman', 'occupation', 'application', 'appointment', 'computing'], 
    'school' : ['school', 'education', 'university', 'academy', 'college', 'teacher', 'classroom', 'grammar school', 'student', 'primary school', 'secondary school', 'educate', 'vocational school', 'seminary', 'schoolhouse', 'kindergarten', 'educational institution', 'conservatory', 'day school', 'shoal', 'public school', 'elementary school', 'private school', 'middle school', 'graduate school', 'curriculum', 'academic', 'gymnasium', 'schoolteacher', 'lyceum', 'deconstructivism', 'schoolroom', 'institution', 'survey', 'learn', 'read', 'research', 'report', 'analyze', 'compary', 'essay', 'major', 'memorizer', 'science', 'learning', 'examination', 'graduation', 'review', 'concentration', 'memorise', 'discipline', 'field of study', 'subject field', 'subject area', 'graduate', 'knowledge', 'study','researchers', 'analysis', 'schoolwork', 'preparation', 'homework', 'coursework', 'tutoring', 'typing', 'teaching', 'prep', 'lesson', 'tutorial', 'basics', 'housework', 'kids', 'math', 'errands', 'assignments', 'preschool', 'tasks', 'exams', 'paperwork', 'mathematics', 'english', 'history', 'literature', 'thesis', 'fail', 'delay', 'language', 'geography', 'course' ]
    }

In [1]:
display(df_journal)

NameError: name 'df_journal' is not defined

In [4]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def lemma1(word):
    return WordNetLemmatizer().lemmatize(word)

def lemma2(word):
    return WordNetLemmatizer().lemmatize(word,'v')

def prepare_text(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2] #check what works best 
    #tokens = [porter.stem(token) for token in tokens] #stemming
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens] #lemmatization 
    tokens = [lemma2(token) for token in tokens] #lemmatization verbs
    return tokens

def prepare_topics(topics):
    for topic in topics: 
        prepared_words = [] 
        for string in topics[topic]:
            string = prepare_text(string)
            prepared_words.append(string)
            res = []
            [res.append(x) for x in prepared_words if x not in res] 
            res = [' '.join(map(str, i)) for i in res]
        topics[topic] = res
    return topics 

def prepare_tag(tag): 
    tokens = tokenize(tag)
    tokens = [token for token in tokens if len(token) > 2] 
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens]  
    tokens = [lemma2(token) for token in tokens] 
    return tokens

def sentiment_analyzer(sentence):
    score = analyser.polarity_scores(sentence)
    negative_score = score['neg']
    positive_score = score['pos']
    neutral_score = score['neu']
    difference = abs(negative_score - positive_score)
    if neutral_score > 0.8:
        return 'neutral'
    elif difference <0.1:
        return 'neutral'
    elif negative_score > positive_score:
        return 'negative'
    else:
        return 'positive'

def dutch_translator(text):
    result = translator.translate(text, dest='en',  src='nl')
    return result.text

def eng_translator(text):
    result = translator.translate(text, dest='nl',  src='en')
    return result.text

In [12]:
prepared_topics = prepare_topics(topics)
print(prepared_topics)
df_journal['prepared_tag'] = df_journal.apply(lambda row: prepare_tag(row.tag), axis = 1)
display(df_journal)

{'social media': ['social', 'medium', 'network', 'twitter', 'facebook', 'instagram', 'web', 'linkedin', 'google', 'viral', 'google+', 'event', 'website', 'reach', 'telegraph', 'telephone', 'blog', 'internet', 'sociality', 'technoself', 'intranet', 'cyber', 'business', 'sociable', 'activist', 'microblogging', 'socially', 'cybernetic', 'pinterest', 'youtube', 'tumblr', 'cyberspace', 'extranet', 'telnet', 'cybernetics', 'modem', 'communication', 'informatics', 'virtual', 'party', 'picture'], 'body insecurity': ['body insecurity', 'body', 'insecure', 'belly', 'skin', 'waist', 'hair', 'eye', 'lip', 'ear', 'forehead', 'eyebrow', 'chin', 'scar', 'nose', 'look', 'eat', 'boldness', 'beard', 'smile', 'teeth', 'stomach', 'back', '', 'butt', 'frame', 'feature', 'physique', 'bod', 'torso', 'arm', 'head', 'corpse', 'chest', 'shape', 'shoulder', 'form', 'neck', 'structure', 'leg', 'mass', 'weight', 'shaky', 'unattractive', 'unsure', 'anxious', 'jealous', 'unconfident', 'distrustful', 'uncomfortable',

,Unnamed: 0,id,type,tag,description,id_user,gold_label,description_eng,prepared_description,prepared_tag,social_media,body_insecurity,social_anxiety,relationships,work,school,topic,sentiment,check_topics
0,1,3,did,headache,Vanmiddag had ik weer zo veel hofodpijn en toe...,6,school,This afternoon I had so much hofod pain again ...,"['afternoon', 'much', 'hofod', 'pain', 'think'...",[headache],0,2,0,1,2,1,no topic,neutral,False
1,2,4,did,school,Ik moest eigenlijk nog dingen doen voor school...,6,school,I actually had to do things for school such as...,"['actually', 'thing', 'school', 'read', 'chapt...",[school],0,0,1,1,1,12,[school],negative,True
2,3,5,wish,Exam stress,Vandaag had ik tentamen training en in plaats ...,10,school,Today I had exam training and instead of telli...,"['today', 'exam', 'train', 'instead', 'tell', ...","[exam, stress]",0,1,3,0,2,5,[school],neutral,True
3,4,6,wish,Work,Vandaag had ik een situatie op werk waar ik mi...,11,work,Today I had a situation at work where I heard ...,"['today', 'situation', 'work', 'hear', 'name',...",[work],0,4,12,4,15,0,[work],neutral,True
4,7,9,did,#body image,Ik had een leuke outfit aan en voelde me goed ...,13,body insecurity,I was wearing a nice outfit and felt good when...,"['wear', 'nice', 'outfit', 'felt', 'good', 'pu...","[body, image]",0,15,1,2,2,0,[body insecurity],neutral,True
5,9,11,did,Work,Vandaag stuitte ik weer op een fout op werk. D...,11,work,Today I ran into another mistake at work. Not ...,"['today', 'run', 'another', 'mistake', 'work',...",[work],1,0,3,5,11,0,[work],neutral,True
6,11,13,wish,es,"Spanning na het afronden van de therapie, het ...",13,no topic,"Tension after finishing the therapy, I felt li...","['tension', 'finish', 'therapy', 'felt', 'like...",[],0,2,1,3,0,0,[relationships],neutral,False
7,12,14,did,inner child,Ik merk dat ik vandaag heel erg stemmen in mij...,12,relationships,I notice that today I heard very many voices i...,"['notice', 'today', 'hear', 'many', 'voice', '...","[inner, child]",0,5,1,5,1,0,"[body insecurity, relationships]",neutral,True
8,14,15,wish,Sad,De foto's van mijn bachelor uitreiking waren t...,24,no topic,I thought the photos of my bachelor's ceremony...,"['think', 'photo', 'bachelor', 'ceremony', 'tu...",[sad],0,2,1,0,1,0,no topic,neutral,True
9,15,16,wish,Skating weekend,Ik had zelfcompassie willen gebruiken; afgelop...,20,social anxiety,I would have liked to use self-compassion; las...,"['would', 'like', 'use', 'self', 'compassion',...","[skate, weekend]",1,3,3,2,1,0,"[body insecurity, social anxiety]",positive,True


In [6]:
def find_topic_tag(text, tag, prepared_topics):
    sim_scores = [] 
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6

        sim_scores.append(sim_score)
    highest_score = max(sim_scores)
    if highest_score < 3:
        return 'no topic'
    else:
        index_highest_score = list_duplicates_of(sim_scores, highest_score)

        chosen_topics = [] 
        for i in index_highest_score:
            topic =  list(prepared_topics)[i]
            chosen_topics.append(topic.strip()) 
        return chosen_topics
    

def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

def social_media_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[0]

def body_insecurity_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[1]

def social_anxiety_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[2]

def relationships_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[3]

def work_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[4]

def school_score(text, tag, prepared_topics):
    sim_scores = []
    for topic in prepared_topics:
        sim_score = 0    
        for word in prepared_topics[topic]:
            if word in text:
                if word != '':
                    occurences = text.count(word)
                    sim_score += occurences

            if word in tag: 
                if word != '':
                    sim_score += 3
                    
        tokenized_topic = tokenize(topic)
        for word in tokenized_topic:
            if word in tag:
                   sim_score += 6
        sim_scores.append(sim_score)
    return sim_scores[5]

def find_sentiment(text):
    sentiment = sentiment_analyzer(text)
    return sentiment 

def add_topic_score_to_df(prepared_topics, df_journal):
    df_journal['social_media'] = df_journal.apply(lambda row: social_media_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['body_insecurity'] = df_journal.apply(lambda row: body_insecurity_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['social_anxiety'] = df_journal.apply(lambda row: social_anxiety_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['relationships'] = df_journal.apply(lambda row: relationships_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['work'] = df_journal.apply(lambda row: work_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)    
    df_journal['school'] = df_journal.apply(lambda row: school_score(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['topic'] = df_journal.apply(lambda row: find_topic_tag(row.prepared_description, row.prepared_tag, prepared_topics), axis = 1)
    df_journal['sentiment'] = df_journal.apply(lambda row: find_sentiment(row.description_eng), axis = 1)

In [2]:
add_topic_score_to_df(prepared_topics, df_journal)
df_journal['check_topics'] = df_journal.apply(lambda row: row.gold_label in row.topic, axis=1)
score = df_journal['check_topics'].value_counts()
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal)

NameError: name 'add_topic_score_to_df' is not defined

In [3]:
df_journal_topics = df_journal.loc[df_journal['gold_label'] != 'no topic']
score = df_journal_topics['check_topics'].value_counts()
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal_topics))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal_topics))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal_topics)

NameError: name 'df_journal' is not defined

In [4]:
df_journal_topics = df_journal.loc[df_journal['gold_label'] == 'no topic']
score = df_journal_topics['check_topics'].value_counts()
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal_topics))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal_topics))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal_topics)

NameError: name 'df_journal' is not defined

In [6]:
df_journal_topics1 = df_journal_topics.loc[df_journal['gold_label'] == 'school']
score = df_journal_topics1['check_topics'].value_counts()
print(score)
correct = score[1]
incorrect = score[0]
percentage_correct = round(((correct/len(df_journal_topics1))*100), 2)
percentage_incorrect = round(((incorrect/len(df_journal_topics1))*100), 2)
print('The number of correct classifications is:', correct, '. This is', percentage_correct, 'percent of the total amount.', '\nThe number of incorrect classifications is:', incorrect, '. This is', percentage_incorrect, 'percent of the total amount.')
display(df_journal_topics1)

NameError: name 'df_journal_topics' is not defined

In [7]:
df_journal.groupby(by='gold_label').count()

NameError: name 'df_journal' is not defined

In [8]:
#kijkt naar het percentage van de punten per topic en de kans op een topic is even groot als dit percentage (met 20% willekeurig)
def topics_per_user1(df_journal, df_exercises, user_id):
    list1 = []
    list1.append(user_id)
    df_journal = df_journal[df_journal.id_user.isin(list1)]
    topic_df = df_journal[['social_media', 'social_anxiety', 'body_insecurity', 'friendship', 'work', 'school', 'relationship']]
    topics = list(topic_df.columns)
    sum_df = topic_df.sum()
    total = sum_df.sum()
    percentages = []
    for i in sum_df:
        percentage = (i / total * 100)
        percentages.append(percentage)
    percentages1 = np.cumsum(percentages)
    dicts = dict(zip(topics, percentages1))    
    sorted_dict = dict(sorted(dicts.items(), key=lambda item: item[1]))
    random1 = randint(0,100)
    if random1 < list(sorted_dict.values())[0]: 
        chosen_topic1 = list(sorted_dict.keys())[0]
    elif random1 < list(sorted_dict.values())[1]: 
        chosen_topic1 = list(sorted_dict.keys())[1]
    elif random1 < list(sorted_dict.values())[2]: 
        chosen_topic1 = list(sorted_dict.keys())[2]
    elif random1 < list(sorted_dict.values())[3]: 
        chosen_topic1 = list(sorted_dict.keys())[3]
    elif random1 < list(sorted_dict.values())[4]: 
        chosen_topic1 = list(sorted_dict.keys())[4]
    elif random1 < list(sorted_dict.values())[5]: 
        chosen_topic1 = list(sorted_dict.keys())[5]
    elif random1 < list(sorted_dict.values())[6]: 
        chosen_topic1 = list(sorted_dict.keys())[6]
    else: 
        chosen_topic1 = random.choice(topics)
    chosen_question_id = return_questions1(df_exercises, user_id, chosen_topic1)
    return chosen_question_id

def pick_random_question(all_ids, likely_ids):
    random_nr = random.uniform(0, 1)
    if random_nr > 0.8:
        chosen_question = random.choice(all_ids)
    else:
        chosen_question = random.choice(likely_ids)
    return chosen_question 

def return_questions1(df_exercises, user_id, chosen_topic1):
    df = df_exercises[df_exercises.Topic == chosen_topic1]
    all_ids = list(df_exercises['ID'])
    likely_ids = list(df['ID'])
    chosen_question_id = pick_random_question(all_ids, likely_ids)
    return(chosen_question_id)

def question_for_user(user_id, df_journal, df_exercises):
    chosen_question_id = topics_per_user1(df_journal, df_exercises, user_id)
    print(chosen_question_id)

question_for_user(11, df_journal, df_exercises)

NameError: name 'df_journal' is not defined